In [1]:
pip install tensorflow pillow matplotlib opencv-python-headless lime


Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.applications import InceptionV3, Xception, DenseNet121
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import lime
from lime import lime_image
import cv2
import matplotlib.pyplot as plt

In [5]:
train_dir = "/Users/owaiskhan/Downloads/Lung Disease Dataset/train"
val_dir = "/Users/owaiskhan/Downloads/Lung Disease Dataset/val"
test_dir = "/Users/owaiskhan/Downloads/Lung Disease Dataset/test"

In [7]:
# Model input parameters
IMG_SIZE = (299, 299)
BATCH_SIZE = 32
LEARNING_RATE = 1e-4
EPOCHS = 31
CLASS_LABELS = ['Bacterial Pneumonia', 'Corona Virus Disease', 'Normal', 'Tuberculosis', 'Viral Pneumonia']


In [9]:
# Data Preprocessing and Augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=10,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)

val_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_data = train_datagen.flow_from_directory(
    train_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical'
)
val_data = val_datagen.flow_from_directory(
    val_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical'
)
test_data = test_datagen.flow_from_directory(
    test_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical', shuffle=False
)


Found 6054 images belonging to 5 classes.
Found 2016 images belonging to 5 classes.
Found 2025 images belonging to 5 classes.


In [11]:
# Build and Compile Models
def build_model(base_model):
    x = base_model.output
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(len(CLASS_LABELS), activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

In [13]:
# Fine-tune Xception and DenseNet121
xception_base = Xception(weights='imagenet', include_top=False, input_shape=(299, 299, 3))
xception_model = build_model(xception_base)
xception_model.compile(optimizer=Adam(learning_rate=LEARNING_RATE), loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
from tensorflow.keras.applications import Xception
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import mixed_precision

# Enable mixed precision policy
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

# Use Xception model with input size of 299x299
model = Xception(weights="imagenet", include_top=False, input_shape=(299, 299, 3))
model.trainable = False  # Freeze base layers initially

# Add custom classification head
x = tf.keras.layers.GlobalAveragePooling2D()(model.output)
x = tf.keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(5, activation="softmax")(x)  # 5 classes for lung diseases

fine_tuned_model = tf.keras.Model(inputs=model.input, outputs=output)

# Compile with mixed precision optimizer
fine_tuned_model.compile(optimizer=Adam(1e-4), loss="categorical_crossentropy", metrics=["accuracy"])

# Set up callbacks
checkpoint = ModelCheckpoint("best_lung_disease_model.keras", monitor="val_accuracy", save_best_only=True, mode="max")
early_stop = EarlyStopping(monitor="val_accuracy", patience=5)

# Train with optimized settings
history = fine_tuned_model.fit(
    train_data,
    validation_data=val_data,
    epochs=20,
    batch_size=32,
    callbacks=[checkpoint, early_stop]
)
